In [1]:
%%capture
#%pip install llama-index cohere llama-index-embeddings-cohere qdrant-client llama-index-vector-stores-qdrant llama-index-llms-cohere
%pip install llama-index mistralai llama-index-embeddings-mistralai qdrant-client llama-index-vector-stores-qdrant llama-index-llms-mistralai

In [2]:
import os
import sys
from getpass import getpass
import nest_asyncio

from IPython.display import Markdown, display

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

sys.path.append('../helpers')

from utils import setup_llm, setup_embed_model, setup_vector_store

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/conda/envs/llama/lib/python3.13/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
#CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")
MISTRAL_API_KEY = os.environ['MISTRAL_API_KEY']

In [ ]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [4]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [5]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [ ]:
from llama_index.core.settings import Settings
from utils import setup_llm, setup_embed_model, setup_vector_store

COLLECTION_NAME = "words-of-the-senpai-naive"

setup_llm(
    provider="cohere", 
    model="command-r-plus", 
    api_key=CO_API_KEY
    )

setup_embed_model(
    provider="cohere",
    api_key=CO_API_KEY
    )

vector_store = setup_vector_store(QDRANT_URL, QDRANT_API_KEY, COLLECTION_NAME)

In [18]:
from llama_index.core.settings import Settings
from utils import setup_llm, setup_embed_model, setup_vector_store

COLLECTION_NAME = "words-of-the-senpai-naive"

setup_llm(
    provider="mistral", 
    model="mistral-small-latest", 
    api_key=MISTRAL_API_KEY
    )

setup_embed_model(
    provider="mistral",
    api_key=MISTRAL_API_KEY
    )

vector_store = setup_vector_store(QDRANT_URL, QDRANT_API_KEY, COLLECTION_NAME)

Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


In [7]:
Settings.llm

MistralAI(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7266061dae90>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x726608769ee0>, completion_to_prompt=<function default_completion_to_prompt at 0x72660880d120>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='open-mistral-nemo', temperature=0.1, max_tokens=512, timeout=120.0, max_retries=5, random_seed=None, additional_kwargs={})

In [8]:
Settings.embed_model


MistralAIEmbedding(model_name='mistral-embed', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7266061dae90>, num_workers=None)

# Load Docstore

In [9]:
from utils import get_documents_from_docstore

documents = get_documents_from_docstore("../data/words-of-the-senpais")

In [10]:
documents[0].__dict__

{'id_': '68e7881a-d87e-4cda-845f-7eb5e88d84c8',
 'embedding': None,
 'metadata': {'page_number': 0,
  'file_name': '../data/almanack_of_naval_ravikant.pdf',
  'title': 'The Almanack of Naval Ravikant',
  'author': 'Naval Ravikant'},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {},
 'text': 'UNDERSTAND HOW WEALTH IS CREATED I like to think that if I lost all my money and you dropped me on a random street in any English-speaking country, within five or ten years Id be wealthy again because its just a skillset Ive developed that anyone can develop. Its not really about hard work. You can work in a restaurant eighty hours a week, and youre not going to get rich. Getting rich is about knowing what to do, who to do it with, and when to do it. It is much more about understanding than purely hard work. Yes, hard work matters, and you cant skimp on it. But it has to be directed in the right way. If you dont know yet what you should work on, the most 

# Ingest to VectorDatabase

In [ ]:
from llama_index.core.constants import DEFAULT_CHUNK_SIZE
from llama_index.core.node_parser.text import SentenceSplitter
from llama_index.core import StorageContext
from utils import ingest

print(f"This is the chunk size: {DEFAULT_CHUNK_SIZE}")

transforms = [
    SentenceSplitter(chunk_size=DEFAULT_CHUNK_SIZE), 
    Settings.embed_model
    ]

nodes = ingest(
    documents=documents,
    transformations=transforms,
    vector_store=vector_store,
)

In [12]:
from llama_index.core.constants import DEFAULT_CHUNK_SIZE
from llama_index.core.node_parser.text import SentenceSplitter
from llama_index.core import StorageContext
from utils import ingest
import time

print(f"This is the chunk size: {DEFAULT_CHUNK_SIZE}")

transforms = [
    SentenceSplitter(chunk_size=DEFAULT_CHUNK_SIZE), 
    Settings.embed_model
]

def process_documents_with_delay(documents, transformations, vector_store):
    for doc in documents:
        # Process one document
        nodes = ingest(
            documents=[doc],  # Process single document instead of all at once
            transformations=transformations,
            vector_store=vector_store,
        )
        
        # Wait 1 seconds before processing the next document
        #print("Waiting 5 seconds before next document...")
        time.sleep(2)
        
        yield nodes

# Use the rate-limited version
nodes = list(process_documents_with_delay(
    documents=documents,
    transformations=transforms,
    vector_store=vector_store,
))

This is the chunk size: 1024


Collection words-of-the-senpai-naive already exists, skipping collection creation.
Collection words-of-the-senpai-naive already exists, skipping collection creation.
Collection words-of-the-senpai-naive already exists, skipping collection creation.
Collection words-of-the-senpai-naive already exists, skipping collection creation.
Collection words-of-the-senpai-naive already exists, skipping collection creation.
Collection words-of-the-senpai-naive already exists, skipping collection creation.
Collection words-of-the-senpai-naive already exists, skipping collection creation.
Collection words-of-the-senpai-naive already exists, skipping collection creation.
Collection words-of-the-senpai-naive already exists, skipping collection creation.
Collection words-of-the-senpai-naive already exists, skipping collection creation.
Collection words-of-the-senpai-naive already exists, skipping collection creation.
Collection words-of-the-senpai-naive already exists, skipping collection creation.
Coll

# Build index over vector database


In [19]:
from utils import create_index, create_query_engine

storage_context = StorageContext.from_defaults(
    vector_store=vector_store
    )

index = create_index(
    from_where="vector_store",
    embed_model=Settings.embed_model, 
    vector_store=vector_store, 
    # storage_context=storage_context
    )

query_engine = create_query_engine(
    index=index, 
    mode="query",
    # llm=Settings.llm
    )

# Query Pipelines



In [20]:
from utils import create_query_pipeline

from llama_index.core.query_pipeline import InputComponent

input_component = InputComponent()

chain = [input_component, query_engine]

query_pipeline = create_query_pipeline(chain)

# Query

In [22]:
response_1 = query_pipeline.run(input="What mental models, principles or philosophies are most valuable for making important life decisions and navigating an uncertain world?")

> Running module 863775ed-8dc2-4284-b379-83081914079e with input: 
input: What mental models, principles or philosophies are most valuable for making important life decisions and navigating an uncertain world?

> Running module f46a5e1c-92b1-4ae6-870a-8402e447283a with input: 
input: What mental models, principles or philosophies are most valuable for making important life decisions and navigating an uncertain world?



In [23]:
print(response_1)

Mental models, principles, and philosophies that are most valuable for making important life decisions and navigating an uncertain world include those derived from evolution, game theory, and the insights of notable figures like Charlie Munger, Nassim Taleb, and Benjamin Franklin. These models help in understanding and predicting outcomes by providing a framework for decision-making that goes beyond specific circumstances. They serve as compact ways to recall deep-seated principles and underlying experiences, making them essential tools for rational and accurate decision-making.


In [24]:
response_1.source_nodes[0].__dict__

{'node': TextNode(id_='ebe1c951-a925-452e-940b-4f6ee02bb9a3', embedding=None, metadata={'page_number': 76, 'file_name': '../data/almanack_of_naval_ravikant.pdf', 'title': 'The Almanack of Naval Ravikant', 'author': 'Naval Ravikant'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a3522937-338d-4afb-8263-d000502b1fd7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_number': 76, 'file_name': '../data/almanack_of_naval_ravikant.pdf', 'title': 'The Almanack of Naval Ravikant', 'author': 'Naval Ravikant'}, hash='3a49e6f7304356b26fde4a4bdd5ca2747beef954f087319476965d27832e485c')}, text='technology and large workforces and capital, our decisions are leveraged more and more. If you can be more right and more rational, youre going to get nonlinear returns in your life. I love the blog Farnam Street because it really focuses on helping you be more accurate, an overall better decision-maker. Decision-m

In [29]:
response_2 = query_pipeline.run(input="What is the essence of a life well-lived, and what are the key components of a good life that everyone should strive for?")
print(response_2)

> Running module 863775ed-8dc2-4284-b379-83081914079e with input: 
input: What is the essence of a life well-lived, and what are the key components of a good life that everyone should strive for?

> Running module f46a5e1c-92b1-4ae6-870a-8402e447283a with input: 
input: What is the essence of a life well-lived, and what are the key components of a good life that everyone should strive for?

A life well-lived is a process of continuous growth and self-expression, not a static state of being. It involves moving in any direction with psychological freedom, and it is a direction selected by the total organism. The good life is a process of discovery and understanding, not a destination. It is a life of harmony, unity, contentment, tranquility, constancy, enlightenment, peace, and long life. It is a life of simplicity, where profit, cleverness, selfishness, and desires are reduced. It is a life that is understood from moment to moment, and it is a life that is enjoyed in the process of plan

In [28]:
response_3 = query_pipeline.run(input="What is the essence of a life well-lived, and what are the key components of a good life that everyone should strive for?")
print(response_3)

> Running module 863775ed-8dc2-4284-b379-83081914079e with input: 
input: What is the essence of a life well-lived, and what are the key components of a good life that everyone should strive for?

> Running module f46a5e1c-92b1-4ae6-870a-8402e447283a with input: 
input: What is the essence of a life well-lived, and what are the key components of a good life that everyone should strive for?

A life well-lived is a process of continuous growth and self-expression, not a static state of being. It is about moving in a direction chosen by the whole self, with psychological freedom. Key components of a good life include:

- **Harmony and Unity**: Living in harmony with oneself and others, and understanding the oneness of all life.
- **Contentment and Tranquility**: Finding contentment and tranquility in simplicity, discarding profit, cleverness, selfishness, and excessive desires.
- **Enlightenment and Peace**: Pursuing enlightenment and peace, understanding life from moment to moment, and e

In [ ]:
response_1 = query_pipeline.run(input="What role does luck play in achieving outsized success and wealth creation? Is capitalizing on luck a skill that can be cultivated?")

In [ ]:
response_1 = query_pipeline.run(input="What are some commonalities between Naval Ravikant and Nassim Taleb's thinking on luck?")